In [31]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

In [32]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 5

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [33]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [34]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [35]:

# Define the MobileNet base model
base_model = MobileNet(
    input_shape=(img_height, img_width, 3),  # Adjust input shape
    include_top=False,  # Exclude the fully-connected layers
    weights='imagenet'  # Pre-trained on ImageNet
)

# Freeze the base model layers
base_model.trainable = False


In [36]:

# Create the classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(14, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [37]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




Epoch 1/5
235/235 [==============================] - 24s 78ms/step - loss: 1.8135 - accuracy: 0.4055 - val_loss: 1.3323 - val_accuracy: 0.5537
Epoch 2/5
235/235 [==============================] - 16s 69ms/step - loss: 1.1342 - accuracy: 0.6542 - val_loss: 1.0257 - val_accuracy: 0.6542
Epoch 3/5
235/235 [==============================] - 16s 70ms/step - loss: 0.9285 - accuracy: 0.7189 - val_loss: 0.8817 - val_accuracy: 0.7033
Epoch 4/5
235/235 [==============================] - 17s 70ms/step - loss: 0.7796 - accuracy: 0.7692 - val_loss: 0.9319 - val_accuracy: 0.6963
Epoch 5/5
235/235 [==============================] - 16s 69ms/step - loss: 0.7680 - accuracy: 0.7802 - val_loss: 0.8408 - val_accuracy: 0.7453


In [38]:
# predicting train dataset 
train_pridiction = model.predict(train_generator)

# true labels 
train_true_labels = train_generator.classes




235/235 [==============================] - 16s 63ms/step


In [39]:
print( train_pridiction.shape)
print( train_true_labels.shape)

#adding the true labels to the pridiction
train_pridiction = np.concatenate((train_pridiction, train_true_labels.reshape(-1,1)), axis=1)
print( train_pridiction.shape)


(1174, 14)
(1174,)
(1174, 15)


In [40]:
#saving predictions in csv file
np.savetxt("../model_saved_files/predictions/EfficientNetV2L_predictions.csv", train_pridiction, delimiter=",")

# Save the model
# model.save("../model_saved_files/models_weights/EfficientNetV2L.h5")

### applying fuzzy using random forest

In [41]:
# applying random forest on the predictions
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df=train_pridiction
X = df[:,:-1]
Y = df[:,-1]


clf = RandomForestClassifier()
clf.fit(X,Y)

RandomForestClassifier()

# TESTING

In [61]:
# applying neural network 
test_prediction = model.predict(test_generator)
test_true_labels = test_generator.classes
test_prediction_true = np.argmax(test_prediction,axis=1)
print(accuracy_score(test_true_labels, test_prediction_true))

86/86 [==============================] - 2s 23ms/step
0.7453271028037384


In [48]:
#applying random forest on the predictions
result=clf.predict(test_prediction)


In [62]:
#printing the classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(accuracy_score(test_true_labels, result))


0.07476635514018691
